In [1]:
import sys
# sys.path.insert(0, '/home/idv-eqs8-pza/IDV_code/Variational_GP/spatial_GP')
import time
import random
import csv
import numpy as np

import scipy.io
from scipy.io import loadmat

import torch
import pickle

from tqdm import tqdm
from tqdm import notebook

import matplotlib
import matplotlib.pyplot as plt

from utils import *

TORCH_DTYPE = torch.float32
# Set the default dtype to float32
torch.set_default_dtype(TORCH_DTYPE)

Using device: cuda:0 (from utils.py)


## Parameters of the training

In [2]:
rand_xtilde = False # If True, xtilde (inducing points) are chosen randomly, if False, xtilde is chosen from the first ntilde images
noise       = False # If True, noise is added to xtilde (used to avoid inducing points being too close to each other in their space)
rescale     = False # If True, X is rescaled between -1 and 1
cellid      = 1   # Choose cell
ntilde      = 50   # Number of xtilde
kernfun     = acosker # Choose kernel function

Nmstep  = 1
Nestep  = 1
Maxiter = 5

## Import the dataset and preprocess


In [3]:
##################
#  Factor 1/2 is removed from acosker
##################

# Set the device and dtype
torch_dtype = torch.float32 # NB: Basically all of the matrices in Spatial_GP have 1.e-7 added to the diagonal, to be changed if we want to use float64
# torch_dtype = torch.float64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
torch.set_default_dtype(torch_dtype)
torch.set_default_device(device)
print(f'Device is: {device}')


    
# Open the .pkl dataset file for reading in binary mode (rb)
with open('/home/idv-eqs8-pza/IDV_code/Variational_GP/spatial_GP/Data/data2_41mixed_tr28.pkl', 'rb') as file:
    # Load the data from the file
    loaded_data = pickle.load(file)
    # loaded_data is a Dataset object from module Data with attributes "images_train, _val, _test" as well as responses

X_train = torch.tensor(loaded_data.images_train).to(device) #shape (2910,108,108,1) where 108 is the number of pixels. 2910 is the amount of training data
X_val   = torch.tensor(loaded_data.images_val).to(device)
X_test  = torch.tensor(loaded_data.images_test).to(device) # shape (30,108,108,1) # nimages, npx, npx

R_train = torch.tensor(loaded_data.responses_train).to(device, dtype=torch_dtype) #shape (2910,41) 2910 is the amount of training data, 41 is the number of cells
R_val   = torch.tensor(loaded_data.responses_val).to(device, dtype=torch_dtype)
R_test   = torch.tensor(loaded_data.responses_test).to(device, dtype=torch_dtype) # shape (30,30,42) 30 repetitions, 30 images, 42 cells

# Concatenate Xtrain and Xval if not using validation set 
X = torch.cat( (X_train, X_val), axis=0,) #shape (3160,108,108,1)
R = torch.cat( (R_train, R_val), axis=0,)
# X = X_train
# R = R_train

# Rescale X to be between -1 and 1
if rescale == True:
    X = (X - X.min()) / (X.max() - X.min()) * 2 - 1

# Choose a cell
r = R[:,cellid] # shape (nt,) where nt is the number of trials

# Reshape images to 1D vector
n_px_side = X.shape[1] # 108   
X = torch.reshape(X, ( X.shape[0], X.shape[1]*X.shape[2])) # shape (n x , 11664)=(nt, nx)
# If X_val is being used, reshape it
# X_val = torch.reshape(X_val, ( X_val.shape[0], X_val.shape[1]*X_val.shape[2])) # shape (n x val, 11664)=(nt, nx)

if rand_xtilde == True:
    torch.manual_seed(2024)
    indices = torch.randint(0, X.shape[0], (ntilde,))
else:
    indices = torch.arange(0,ntilde, dtype=torch.int64)
if noise == True:
    xtilde = X[indices,:] + 1e-7*torch.rand(X[indices,:].shape)*2 - 1.e-7
else:
    xtilde = X[indices,:]


Device is: cuda:0


In [25]:

# If one wants to compare the hyperparemeters set in Samuele's code:
logsigma_0 = torch.tensor(0) # Samuele's code set the log of sigma

logbetaexpr = fromlogbetasam_to_logbetaexpr( logbetasam=torch.tensor(5.5) )# Logbetaexpr in this code is equal to logbeta in Samuele's code. Samuele's code set logbeta to 5.5

logrhoexpr  = fromlogrhosam_to_logrhoexpr( logrhosam=torch.tensor(5)) 

Amp = torch.tensor(1.0) # Samuele's code set Amp to 1 does not learn it (absent in the code)

theta = {'sigma_0': torch.exp(logsigma_0), 'eps_0x':torch.tensor(0.0), 'eps_0y':torch.tensor(0.0), '-2log2beta': logbetaexpr, '-log2rho2': logrhoexpr, 'Amp': Amp }

for key, value in theta.items():
    theta[key] = value.requires_grad_()

hyperparams_tuple = generate_theta( x=X, r=r, n_px_side=n_px_side, display=True, **theta)


args = {
        'ntilde':  ntilde,
        'Maxiter': Maxiter,
        'Nmstep':  Nmstep,
        'Nestep':  Nestep,
        'kernfun': kernfun,
        'n_px_side': n_px_side,
        'display_hyper': True,
        'display_prog':  True,
        'hyperparams_tuple': hyperparams_tuple,
    }

# Train model
theta, f_params, m, V, C, mask, K_tilde_inv, K_tilde, values_track = varGP(X, r, **args)



updated sigma_0 to 1.0
updated eps_0x to 0.0
updated eps_0y to 0.0
updated -2log2beta to 4.8068528175354
updated -log2rho2 to 4.3068528175354
updated Amp to 1.0
 Before overloading
 Hyperparameters have been SET as  : beta = 0.05856070, rho = 0.02928035
 Samuele hyperparameters           : logbetasam = 4.9822, logrhosam = 7.0617

 After overloading
 Dict of learnable hyperparameters : sigma_0 = 1.00000000, eps_0x = 0.00000000, eps_0y = 0.00000000, -2log2beta = 4.80685282, -log2rho2 = 4.30685282, Amp = 1.00000000
 Hyperparameters from the logexpr  : beta = 0.04520382, rho = 0.08208501
 Samuele hyperparameters           : logbetasam = 5.5000, logrhosam = 5.0000
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/idv-eqs8-pza/anaconda3/envs/pytorch/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_13490/2976481276.py", line 32, in <module>
    theta, f_params, m, V, C, mask, K_tilde_inv, K_tilde, values_track = varGP(X, r, **args)
                                                                         ^^^^^^^^^^^^^^^^^^^
  File "/home/idv-eqs8-pza/IDV_code/Variational_GP/Spatial_GP_repo/utils.py", line 1010, in varGP
    C, mask = localker(nx=x.shape[1], theta=theta, theta_higher_lims=theta_higher_lims, theta_lower_lims=theta_lower_lims, n_px_side=n_px_side, grad=False) #TODO check that the shape is correct for any input shape
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/idv-eqs8-pza/IDV_code/Variational_GP/Spa

In [ ]:

# Save the model
model = {
    'theta': theta,
    'f_params': f_params,
    'm': m,
    'V': V,
    'C': C,
    'mask': mask,
    'K_tilde_inv': K_tilde_inv,
    'K_tilde': K_tilde,
    'cellid': cellid,
    'ntilde': ntilde,
    'Maxiter': Maxiter,
    'Nmstep': Nmstep,
    'Nestep': Nestep,
    'kernfun': kernfun,
    'values_track': values_track,
}

# save_pickle('pietro_model', **model)


torch.set_grad_enabled(False)

# TEMP
# To compare to samuele I have to kep the aplitude of C fixed to 1, he does not have it
theta['Amp'] = 1.0


print(f'f_params: {f_params}')
for key, value in theta.items():
    print(f"{key}: {value:.4f}")

# Predict and test

rtst, R_predicted, r2, sigma_r2 = test(X_test, R_test, xtilde, **model )

# Plot results
fig = plt.figure(figsize=(12, 8))
gs = fig.add_gridspec(5, 5,
            left=0.1, right=0.9, bottom=0.1, top=0.9,
            wspace=0.3, hspace=0.7)
dt = 0.05
time_values = dt * np.arange( len(R_predicted) )
ax = fig.add_subplot(gs[3:, :])
ax.plot(time_values, np.mean(rtst, axis=0) / 0.05, 'k', linewidth=1)

ax.plot(time_values, R_predicted / 0.05, color='red', label='GP')
# ax.errorbar(time_values, R_predicted / 0.05, yerr=np.sqrt(sigma2_f[:,0].cpu()) / 0.05, color='red')
# ax.legend(['data', 'GP'], loc='upper right', fontsize=14)
txt = f'Pietro adjusted r^2 = {r2:.2f} ± {sigma_r2:.2f} Cell: {cellid}'
ax.set_title(f'{txt}')
# ax.set_ylabel('Firing rate (Hz)')
plt.show()
plt.close()


a=1